In [1]:
import mysql.connector as connector
connection=connector.connect(
                             user="<USERNAME>", # use your own
                             password="<password>", # use your own
                            )
cursor = connection.cursor()
cursor.execute("USE little_lemon_db")

In [2]:
#  cursor.execute("DROP TABLE IF EXISTS Employees")
# cursor.execute("DROP TABLE IF EXISTS Bookings")
cursor.execute("SHOW TABLES")
result = cursor.fetchall()
print(result)

[('bookings',), ('employees',), ('menuitems',), ('menus',), ('orders',)]


In [ ]:
'''
create_booking_table = """CREATE TABLE Bookings (
BookingID INT AUTO_INCREMENT,
TableNo INT,
GuestFirstName VARCHAR(100) NOT NULL,
GuestLastName VARCHAR(100) NOT NULL,
BookingSlot TIME NOT NULL,
EmployeeID INT,
PRIMARY KEY (BookingID)
);"""
cursor.execute(create_booking_table)
insert_bookings="""
INSERT INTO Bookings (BookingID, TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
(1, 12, 'Anna','Iversen','19:00:00',1),
(2, 12, 'Joakim', 'Iversen', '19:00:00', 1),
(3, 19, 'Vanessa', 'McCarthy', '15:00:00', 3),
(4, 15, 'Marcos', 'Romero', '17:30:00', 4),
(5, 5, 'Hiroki', 'Yamane', '18:30:00', 2),
(6, 8, 'Diana', 'Pinto', '20:00:00', 5);"""
cursor.execute(insert_bookings)
connection.commit()
'''

In [ ]:
# Little Lemon analysis and sales report

""""
You have gained all the necessary skills and knowledge required 
to interact with the back-end MySQL database from your Python-based front-end application 
via the standard database client MySQL Connector/Python. 

Follow the instructions in the exercises given below and 
create a Jupyter notebook to produce a Little Lemon analysis and sales report. 
You need to submit this report for your peers to review.
"""

In [3]:
# Task 1: Complete the following steps to establish a connection pool:

# To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.
# !pip install mysql-connector-python
from mysql.connector.pooling import MySQLConnectionPool

# To find the information on the error, import the Error class from MySQL Connector/Python.
from mysql.connector import Error

# Define your database configurations as a Python dictionary object called dbconfig; PII removed after running
dbconfig = {
    "database":"little_lemon_db",
    "user" : "<USERNAME>",
    "password" : "<pwd>"
}

# Establish a connection pool [pool_name = pool_b] with two connections. 
# Implement error handling using a try-except block in case the connection fails. 

try:
    pool = MySQLConnectionPool(pool_name = "ll_pool_b",
                           pool_size = 2, #default is 5
                           **dbconfig)
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

The connection pool is created with a name:  ll_pool_b
The pool size is: 2


In [4]:
# Recap:
import mysql.connector as connector
connection=connector.connect(
                             # user="<USERNAME>", # use your own
                             # password="<password>", # use your own
                             user="<USERNAME>", # use your own
                             password="<PWD>", # use your own
                            )
cursor = connection.cursor()
cursor.execute("USE little_lemon_db")

In [5]:
# Recap 2:
cursor.execute("SELECT * FROM Bookings")
result = cursor.fetchall()
print(result)

[(1, 12, 'Anna', 'Iversen', datetime.timedelta(seconds=68400), 1), (2, 12, 'Joakim', 'Iversen', datetime.timedelta(seconds=68400), 1), (3, 19, 'Vanessa', 'McCarthy', datetime.timedelta(seconds=54000), 3), (4, 15, 'Marcos', 'Romero', datetime.timedelta(seconds=63000), 4), (5, 5, 'Hiroki', 'Yamane', datetime.timedelta(seconds=66600), 2), (6, 8, 'Diana', 'Pinto', datetime.timedelta(seconds=72000), 5)]


In [6]:
# Recap 3:
cursor.execute("DESC Bookings")
result = cursor.fetchall()
for i in result:
    print(i) #; drop_query = "DROP TABLE IF EXISTS " + i[0];
    # print("Table: " + i[0])

('BookingID', b'int', 'NO', 'PRI', None, 'auto_increment')
('TableNo', b'int', 'YES', '', None, '')
('GuestFirstName', b'varchar(100)', 'NO', '', None, '')
('GuestLastName', b'varchar(100)', 'NO', '', None, '')
('BookingSlot', b'time', 'NO', '', None, '')
('EmployeeID', b'int', 'YES', '', None, '')


In [7]:
# Task 2:
# Three guests are trying to book dinner slots simultaneously. 
# 1. Get the connections from pool_b and
# 2. insert the following data in the Bookings table:
# Guest 1:
# Table Number: 8
# First Name: Anees
# Last Name: Java
# Booking Time: 18:00
# EmployeeID: 6

# Guest 2:
# Table Number: 5
# First Name: Bald
# Last Name: Vin
# Booking Time: 19:00
# EmployeeID: 6

# Guest 3:
# Table Number: 12
# First Name: Jay
# Last Name: Kon
# Booking Time: 19:30 
# EmployeeID: 6

list_insert = ["(8, 'Anees', 'Java', '18:00:00', 6)", "(5, 'Bald', 'Vin', '19:00:00', 6)", "(12, 'Jay', 'Kon', '19:30:00', 6)"]     

In [8]:
cursor = connection.cursor()
cursor.execute("USE little_lemon_db")
cursor.execute("DESC Bookings")
result = cursor.fetchall()
print(result)

[('BookingID', b'int', 'NO', 'PRI', None, 'auto_increment'), ('TableNo', b'int', 'YES', '', None, ''), ('GuestFirstName', b'varchar(100)', 'NO', '', None, ''), ('GuestLastName', b'varchar(100)', 'NO', '', None, ''), ('BookingSlot', b'time', 'NO', '', None, ''), ('EmployeeID', b'int', 'YES', '', None, '')]


In [9]:
for i in range(len(list_insert)):
    try:
        print("[{}] is connected.\n".format(list_insert[i]))
    except:
        pass
    finally:
        pass

[(8, 'Anees', 'Java', '18:00:00', 6)] is connected.

[(5, 'Bald', 'Vin', '19:00:00', 6)] is connected.

[(12, 'Jay', 'Kon', '19:30:00', 6)] is connected.



In [10]:
for i in range(len(list_insert)):
    try:
        guest_connected_i = pool.get_connection()
        print("[{}] is connected.\n".format(guest_connected_i))
    except:
        # TIP: You need to add a connection to connect the third guest.
        print("Adding new connection in the pool.")
        connection=connector.connect(user = "<USERNAME>", password = "<PWD>")
        pool.add_connection(cnx=connection)
        print("A new connection is added in the pool.\n")
        guest_connected_i = pool.get_connection()
        print("[{}] is connected.\n".format(guest_connected_i))
        cursor = connection.cursor()
        cursor.execute("USE little_lemon_db")
    finally:
        query_i = "INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) VALUES " + list_insert[i]
        cursor.execute(query_i)
        connection.commit()

[<mysql.connector.pooling.PooledMySQLConnection object at 0x000001E731A5BD90>] is connected.

[<mysql.connector.pooling.PooledMySQLConnection object at 0x000001E73185F400>] is connected.

Adding new connection in the pool.
A new connection is added in the pool.

[<mysql.connector.pooling.PooledMySQLConnection object at 0x000001E731A43FA0>] is connected.



In [11]:
# Recap 2:
connection=connector.connect(
                             user="<USERNAME>", # use your own
                             password="<password>", # use your own
                            )
cursor = connection.cursor()
cursor.execute("USE little_lemon_db")
cursor.execute("SELECT * FROM Bookings")
result = cursor.fetchall()
print(result)

[(1, 12, 'Anna', 'Iversen', datetime.timedelta(seconds=68400), 1), (2, 12, 'Joakim', 'Iversen', datetime.timedelta(seconds=68400), 1), (3, 19, 'Vanessa', 'McCarthy', datetime.timedelta(seconds=54000), 3), (4, 15, 'Marcos', 'Romero', datetime.timedelta(seconds=63000), 4), (5, 5, 'Hiroki', 'Yamane', datetime.timedelta(seconds=66600), 2), (6, 8, 'Diana', 'Pinto', datetime.timedelta(seconds=72000), 5), (7, 8, 'Anees', 'Java', datetime.timedelta(seconds=64800), 6), (8, 5, 'Bald', 'Vin', datetime.timedelta(seconds=68400), 6), (9, 12, 'Jay', 'Kon', datetime.timedelta(seconds=70200), 6)]


In [12]:
dir(pool)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cnx_config',
 '_cnx_queue',
 '_config_version',
 '_pool_name',
 '_pool_size',
 '_queue_connection',
 '_remove_connections',
 '_reset_session',
 '_set_pool_name',
 '_set_pool_size',
 'add_connection',
 'get_connection',
 'pool_name',
 'pool_size',
 'reset_session',
 'set_config']

In [12]:
# Return all the connections back to the pool. 
guest_connected_i.close()

#TIP: The pool size is two. However, you have three connected users. 
#You can only return two connections. 
#Returning a third connection will raise a PoolError. 
#Use try-except to print the error message.  

In [13]:
# Testing 
# Turns out only the last connection is closed

for i in range(len(list_insert)):
    try:
        guest_connected_i = pool.get_connection()
        print("[{}] is connected.\n".format(guest_connected_i))
    except:
        # TIP: You need to add a connection to connect the third guest.
        print("Adding new connection in the pool.")
        connection=connector.connect(user = "<USERNAME>", password = "<PWD>")
        pool.add_connection(cnx=connection)
        print("A new connection is added in the pool.\n")
        guest_connected_i = pool.get_connection()
        # print("[{}] is connected.\n".format(guest_connected_i))
        # cursor = connection.cursor()
        # cursor.execute("USE little_lemon_db")
    finally:
        pass
        

[<mysql.connector.pooling.PooledMySQLConnection object at 0x000001E731A5B2B0>] is connected.

Adding new connection in the pool.
A new connection is added in the pool.

Adding new connection in the pool.
A new connection is added in the pool.



In [9]:
# Testing
list_insert = ["(8, 'Anees', 'Java', '18:00:00', 6)", "(5, 'Bald', 'Vin', '19:00:00', 6)", "(12, 'Jay', 'Kon', '19:30:00', 6)"]
for i in list_insert:
    str_i = "INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) VALUES " + i
    print(str_i)

INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) VALUES (8, 'Anees', 'Java', '18:00:00', 6)
INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) VALUES (5, 'Bald', 'Vin', '19:00:00', 6)
INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) VALUES (12, 'Jay', 'Kon', '19:30:00', 6)


In [6]:
# Task 3:
# Create a report containing the following information:


In [15]:
# The name and EmployeeID of the Little Lemon manager.
cursor = connection.cursor()
cursor.execute('SELECT Name, EmployeeID FROM Employees WHERE Role = "Manager"')
result_manager = cursor.fetchall()
print(result_manager)

[('Mario Gollini', 1)]


In [16]:
# The name and role of the employee who receives the highest salary.
cursor.execute("SELECT Name, Role FROM Employees WHERE Annual_Salary = (SELECT MAX(Annual_Salary) from Employees)")
result_WageHighest = cursor.fetchall()
print(result_WageHighest[0])

('Mario Gollini', 'Manager')


In [17]:
# The number of guests booked between 18:00 and 20:00.
cursor.execute("""SELECT COUNT(*)
                FROM Bookings 
                WHERE BookingID IN 
                    (SELECT BookingID 
                    FROM Bookings
                    WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00' )""")
result_EveningCount = cursor.fetchall()
print(result_EveningCount[0])

(7,)


In [18]:
# The full name and BookingID of all guests 
# waiting to be seated with the receptionist 
# in sorted order with respect to their BookingSlot.
cursor.execute("""WITH 
                    cte_1 AS 
                        (SELECT CONCAT(GuestFirstName, ' ', GuestLastName) AS FullName, BookingID, BookingSlot
                        FROM Bookings 
                        WHERE EmployeeID = (SELECT EmployeeID 
                            FROM Employees
                            WHERE Role = 'Receptionist' )
                            )
                    SELECT FullName, BookingID
                    FROM cte_1
                    ORDER BY BookingSlot""")
result_ToBeSeated = cursor.fetchall()
print(result_ToBeSeated[0])

('Anees Java', 7)


In [19]:
# Report of Task 3
print("The name of the Little Lemon manager: ",result_manager[0][0],"and his/her EmployeeID", result_manager[0][1])
print("The name and role of the employee who receives the highest salary: ",result_WageHighest[0][0],"and his/her role is", result_WageHighest[0][1])

print("The number of guests booked between 18:00 and 20:00: ",result_EveningCount[0][0])
print("The guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot are listed as below: ")
for i in range(len(result_ToBeSeated)):
    print("Guest", i + 1, result_ToBeSeated[i][0], 'with BookingID', result_ToBeSeated[i][1])

The name of the Little Lemon manager:  Mario Gollini and his/her EmployeeID 1
The name and role of the employee who receives the highest salary:  Mario Gollini and his/her role is Manager
The number of guests booked between 18:00 and 20:00:  7
The guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot are listed as below: 
Guest 1 Anees Java with BookingID 7
Guest 2 Bald Vin with BookingID 8
Guest 3 Jay Kon with BookingID 9


In [20]:
# Task 4:
# Create a stored procedure named BasicSalesReport that returns the following statistics:
# Total sales
# Average sale
# Minimum bill paid
# Maximum bill paid

# , AVG(BillAmount) AS AverageSales, MAX(BillAmount) AS MaximumSales, MIN(BillAmount) AS MinimumSales
que_cteProc_BasicSalesReport = """CREATE PROCEDURE BasicSalesReport()
BEGIN
SELECT SUM(BillAmount), AVG(BillAmount) AS AverageSales, MAX(BillAmount) AS MaximumSales, MIN(BillAmount) AS MinimumSales
FROM Orders;
END
"""
cursor.execute(que_cteProc_BasicSalesReport)

In [22]:
# Checking
cursor.callproc("BasicSalesReport")
results = next( cursor.stored_results() )
dataset = results.fetchall()
for i in dataset:
    print(i)

(Decimal('243'), Decimal('48.6000'), 86, 37)


In [23]:
# Report of Task 4
print('Total sales:', float(dataset[0][0]))
print('Average sales:', float(dataset[0][1]))
print('Minimum bill paid:', (dataset[0][3]))
print('Maximum bill paid:', (dataset[0][2]))

Total sales: 243.0
Average sales: 48.6
Minimum bill paid: 37
Maximum bill paid: 86


In [25]:
# Task 5:
# The output should be as follows:
# [BookingSlot]
# [Guest_name]
# [Assigned to: Employee Name [Employee Role]]

# Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen 
# to notify their chefs which orders are due next. To complete this task, carry out the following steps:

# STEP 1: Get a connection from the pool.
guest_connected_i.close()
connected_Notify = pool.get_connection()
print("[{}] is connected.\n".format(connected_Notify))

[<mysql.connector.pooling.PooledMySQLConnection object at 0x000001E731875070>] is connected.



In [26]:
# STEP 2: Create a buffered cursor.
buffered_cursor = connected_Notify.cursor(buffered = True)

# STEP 3: Combine the data from the Bookings and the Employee tables. 
# Sort the retrieved records in ascending order. 
# Then display the information of the first three guests. 
que_Notify = """SELECT Bookings.BookingSlot, Bookings.GuestFirstName, Bookings.GuestLastName,
        Employees.Role, Employees.Name
        FROM Bookings
        INNER JOIN Employees
        ON Bookings.EmployeeID = Employees.EmployeeID
        ORDER BY Bookings.BookingSlot"""

buffered_cursor.execute(que_Notify)
result = buffered_cursor.fetchall()
i = 0
for book_i in result: 
    if i == 3:
        break       
    print(book_i[0])
    print(book_i[1] + ' ' + book_i[2])
    print("Assigned to: " + book_i[4] + "[" + book_i[3] + "]")
    i += 1

15:00:00
Vanessa McCarthy
Assigned to: Giorgos Dioudis[Head Chef]
17:30:00
Marcos Romero
Assigned to: Fatma Kaya[Assistant Chef]
18:00:00
Anees Java
Assigned to: John Millar[Receptionist]


In [27]:
# STEP 4: Returned the connection back to the pool.
connected_Notify.close()

In [76]:
type(connected_Notify)

mysql.connector.pooling.PooledMySQLConnection

In [28]:
try:
    isConnected = connected_Notify.is_connected()
    print('still connected?: ', isConnected)
except:
    print('connection is returned back to the pool')

connection is returned back to the pool


In [45]:
cursor.fetchall()

[(1, 12, 1, 1, 86, 2),
 (2, 19, 2, 2, 37, 1),
 (3, 15, 2, 3, 37, 1),
 (4, 5, 3, 4, 40, 1),
 (5, 8, 1, 5, 43, 1)]

In [46]:
# cursor.execute("SELECT * FROM Orders")
cursor.callproc("BasicSalesReport")
results = next( cursor.stored_results() )

In [47]:
for i in results:
    print(i)

(Decimal('243'), Decimal('48.6000'), 86, 37)


In [48]:
cursor.column_names

('OrderID', 'TableNo', 'MenuID', 'BookingID', 'BillAmount', 'Quantity')

In [49]:
cursor.execute("SHOW PROCEDURE STATUS WHERE Db = 'little_lemon_db'")
results = cursor.fetchall()
for i in results:
    print(i)

('little_lemon_db', 'BasicSalesReport', 'PROCEDURE', 'dmtwong@%', datetime.datetime(2024, 11, 20, 12, 19, 7), datetime.datetime(2024, 11, 20, 12, 19, 7), 'DEFINER', b'', 'utf8mb4', 'utf8mb4_0900_ai_ci', 'utf8mb4_0900_ai_ci')
('little_lemon_db', 'GuestStatus', 'PROCEDURE', 'dmtwong@%', datetime.datetime(2024, 11, 22, 16, 59, 14), datetime.datetime(2024, 11, 22, 16, 59, 14), 'DEFINER', b'', 'utf8mb4', 'utf8mb4_0900_ai_ci', 'utf8mb4_0900_ai_ci')
('little_lemon_db', 'PeakHours', 'PROCEDURE', 'dmtwong@%', datetime.datetime(2024, 11, 19, 15, 45, 46), datetime.datetime(2024, 11, 19, 15, 45, 46), 'DEFINER', b'', 'utf8mb4', 'utf8mb4_0900_ai_ci', 'utf8mb4_0900_ai_ci')
